In [ ]:
import random

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas
import pandas as pd
import seaborn as sns
import yaml
from matplotlib.colors import ListedColormap
from sklearn import metrics
from sklearn.datasets import make_circles, make_blobs, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree


SEED = 314159
TRAIN_TEST_SPLIT = 0.80

data_path = "D:\data\machine_learning"

# Решающие деревья

Решающее дерево предсказывает значение целевой переменной с помощью применения последовательности простых правил. И хотя обобщающая способность решающих деревьев невелика, его все равно часто используют из-за легкой интерпретации. К тому же, решающие деревья успешно объединяются в ансамбли, что решает часть проблем.

В целом, решающие деревья (в основном как части ансамблей) довольно успешно используются. Вопрос: какие еще есть плюсы у решающих деревьев?

Однако решающие деревья также требуют аккуратности. Какая самая простая проблема приходит на ум?
Эту и другие сложности рассмотрим в этом ноутбуке.


Рассмотрим набор данных о качестве вина на основе различных химических показателей. Есть всего 6 значений качества, поэтому задачу проще всего решать классификацией.

In [ ]:
df = pd.read_csv(data_path+'/'+"winequality-red.csv")

In [ ]:
df.describe()

# Разделяющие поверхности

Рассмотрим то, как дерево решений строит разделяющие поверхности. Для начала снова получим обучающие и тестовые данные.

In [ ]:
df_major = df[df["quality"].isin([5,6])]
print("Length of filtered data is", len(df_major))
X = df_major.drop('quality', axis=1)
y = df_major['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

Воспользуемся встроенным функционалом scikit-learn. К сожалению, он позволяет строить только модели для двух входных переменных, поэтому выберем пару переменных поинформативнее.

In [ ]:
# Choosing the first 2 columns for the plot
features = ['volatile acidity', 'alcohol' ]
X_train_cols = X_train.loc[:, features ]
# Creating and fitting the tree classifier
classifier = # your code


In [ ]:
def plot_boundary(
        classifier, data: pd.DataFrame, features: list[str], y: pd.DataFrame | pd.Series | None = None
) -> None:
    # Plotting the tree boundaries
    disp = DecisionBoundaryDisplay.from_estimator(classifier,
                                                  data.loc[:, features],
                                                  response_method="predict_proba",
                                                  xlabel=features[0], ylabel=features[1],
                                                  alpha=0.5,
                                                  cmap=plt.cm.coolwarm,
                                                  grid_resolution=500)

    # Plotting the data points
    # your code. You can use disp.ax_ to access the axis

    plt.title(f"Decision surface for {classifier.__class__.__name__} trained on {features[0]} and {features[1]}")
    plt.show()

In [ ]:
# plot boundary for train data

Отличается ли поверхность для тестового множества?

In [ ]:
X_test_cols = X_test.loc[:, features ]
#  plot boundary for test data

На таком графике может быть хорошо заметно влияние даже одного примера на принятие решения.

In [ ]:
# Choosing the first 2 columns for the plot
features = ['volatile acidity', 'alcohol' ]
X_train_cols = X_train.copy()[:][features]
# change one sample
# Creating and fitting the tree classifier
classifier = DecisionTreeClassifier(max_depth=6,
                                    random_state=SEED).fit(X_train_cols, y_train)

plot_boundary(classifier, X_train_cols, features, y_train)

Решающие деревья часто совмещают с PCA. Это мешает интерпретируемости, но часто достаточно полезно.

In [ ]:
from sklearn.decomposition import PCA
pca =  # your code
X_train_pca = # your code
X_test_pca = # your code

In [ ]:
features = ["x1", "x2"]
classifier = DecisionTreeClassifier(max_depth=6,
                                    random_state=SEED).fit(X_train_pca, y_train)

plot_boundary(classifier, X_train_pca, features, y_train)

In [ ]:
# your code for test dataset

Как видно, границы принятия решений довольно "хаотичны", что говорит о явном переобучении. Давайте оценим, как будут влиять на них различные гиперпараметры.

Для начала рассмотрим глубину дерева.

In [ ]:
# your code - change max depth of the tree and plot the boundaries

In [ ]:
# your code - change max depth of the tree and plot the boundaries

In [ ]:
# your code - change other parameters of the tree and plot the boundaries

Каким будет оптимальный набор гиперпараметров?

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
    # set params space
}

In [ ]:
classifier = DecisionTreeClassifier(max_depth=4,
                                    random_state=SEED)
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           cv=4, n_jobs=-1, verbose=1, scoring = "accuracy").fit(X_train_pca, y_train)

In [ ]:
# print best parameters

In [ ]:
# plot boundaries for best parameters

In [ ]:
# run grid search for previosly fixed depth. Can we achieve similar results using that setting?

In [ ]:
# plot the decision boundary

Другая опция, как можно автоматизировать подбор гиперпараметров - Randomized Search. В отличие от Grid Search, этот метод выбирает параметры не по сетке, а случайно. Причем показано, что при равном бютжете на оптимизацию гиперпараметров он работает даже лучше, чем Grid Search. Посмотрим, будет ли выбор разный.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# run randomized search cv

In [ ]:
# train classifier with new parameters

Рассмотрим и важность выбора сплиттера. Так, DecisionTree позволяет выбирать между random и best. Random отличается тем, что вместо проверки всех возможных порогов разделения по признаку проверяется один случайный порог, взятый из равномерного распределенния между минимумом и максимумом признака.

In [ ]:
X_gen, y_gen = make_classification(n_samples=500, n_features=2000, n_informative=600,
                           n_repeated=0, n_classes=2,
                           n_clusters_per_class=1,
                           class_sep=0.9, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_gen, y_gen, test_size=0.2, random_state=SEED)

In [ ]:
classifier = DecisionTreeClassifier(min_samples_leaf=10,
                                    random_state=SEED).fit(X_train, y_train)
print("Importances", np.sort(classifier.feature_importances_))
print("Depth", classifier.get_depth())
print("Num leaves", classifier.get_n_leaves())
print("Train score is: ", classifier.score(X_train, y_train))
print("Test score is: ", classifier.score(X_test, y_test))

In [ ]:
# train classifier and get the numbers for other split